# 🎵 DTLN Multi-Method Audio Evaluation
**Comprehensive Comparison of 4 Audio Processing Methods**

Interactive notebook for noise suppression experiments comparing DTLN neural network with traditional DSP methods.

## 🎯 Four Evaluation Methods:

1. **Deterministic (DTLN)** - Fixed uploaded noise + DTLN neural network
2. **Stochastic (DTLN)** - Random synthetic noise + DTLN neural network
3. **Traditional-Manual (DSP)** - Spectral Subtraction & Wiener Filter (research-based)
4. **Traditional-Library (DSP)** - noisereduce library (industry standard)

## 📊 Evaluation Features:

- **SNR Range:** -5, 0, 5, 10 dB (4 levels)
- **Metrics:** STOI, PESQ, MSE, MRE with baseline comparison
- **Noise Types:** Gaussian, White, Mixed
- **Fair Comparison:** All methods use blind noise estimation (no ground truth leakage)
- **Comprehensive Visualization:** Waveforms, spectrograms, metrics charts
- **Excel Export:** Multi-sheet results with statistics

**Quick Start:**
1. Install packages → 2. Upload models & audio → 3. Run experiments → 4. Download results

In [ ]:
#@title 📦 Step 1: Install Required Packages { display-mode: "form" }
#@markdown This cell installs all necessary Python packages for the evaluation.

# Install Required Packages
!pip install -q numpy scipy librosa soundfile onnxruntime matplotlib pystoi pesq openpyxl pandas noisereduce

import numpy as np
import soundfile as sf
import librosa
import librosa.display
import matplotlib.pyplot as plt
import onnxruntime
from scipy import signal
from pystoi import stoi
from pesq import pesq
import os
import shutil
from google.colab import files
from IPython.display import display, Audio
import time
import warnings
import pandas as pd
import noisereduce as nr
warnings.filterwarnings('ignore')

# Setup directories
os.makedirs('pretrained_model', exist_ok=True)
os.makedirs('uploads', exist_ok=True)
os.makedirs('outputs', exist_ok=True)
os.makedirs('results/audio', exist_ok=True)
os.makedirs('results/spectrograms', exist_ok=True)
os.makedirs('results/metrics', exist_ok=True)

print("✅ All packages installed successfully!")
print("✅ Directory structure created")

In [ ]:
#@title 🤖 Step 2: Download DTLN Models { display-mode: "form" }
#@markdown Auto-download pre-trained DTLN models from GitHub repository.

# Auto-download DTLN Models
import urllib.request
import ssl

def download_dtln_models():
    models = {
        'model_1.onnx': 'https://github.com/breizhn/DTLN/raw/master/pretrained_model/model_1.onnx',
        'model_2.onnx': 'https://github.com/breizhn/DTLN/raw/master/pretrained_model/model_2.onnx'
    }
    
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
    
    print("📥 Downloading DTLN models from GitHub...")
    
    for model_name, url in models.items():
        model_path = f'pretrained_model/{model_name}'
        
        if os.path.exists(model_path):
            print(f"✓ {model_name} already exists")
            continue
        
        try:
            print(f"⬇️  Downloading {model_name}...", end=' ')
            urllib.request.urlretrieve(url, model_path)
            file_size = os.path.getsize(model_path) / (1024 * 1024)
            print(f"✅ Done ({file_size:.2f} MB)")
        except Exception as e:
            print(f"❌ Failed: {str(e)}")
            return False
    
    return True

# Download models
download_success = download_dtln_models()

if not download_success:
    print("\n⚠️  Auto-download failed. Please upload models manually.")
    uploaded = files.upload()
    for filename in uploaded.keys():
        shutil.move(filename, f'pretrained_model/{filename}')

# Load models
if os.path.exists('pretrained_model/model_1.onnx') and os.path.exists('pretrained_model/model_2.onnx'):
    BLOCK_LEN = 512
    BLOCK_SHIFT = 128
    SAMPLE_RATE = 16000
    
    print("\n⚙️  Loading ONNX models...")
    interpreter_1 = onnxruntime.InferenceSession('pretrained_model/model_1.onnx')
    interpreter_2 = onnxruntime.InferenceSession('pretrained_model/model_2.onnx')
    model_input_names_1 = [inp.name for inp in interpreter_1.get_inputs()]
    model_input_names_2 = [inp.name for inp in interpreter_2.get_inputs()]
    model_inputs_1 = {inp.name: np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32) for inp in interpreter_1.get_inputs()}
    model_inputs_2 = {inp.name: np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32) for inp in interpreter_2.get_inputs()}
    
    print(f"✅ DTLN models loaded successfully!")
    print(f"   Configuration: BLOCK_LEN={BLOCK_LEN}, BLOCK_SHIFT={BLOCK_SHIFT}, SAMPLE_RATE={SAMPLE_RATE}Hz")
else:
    raise FileNotFoundError("DTLN model files are missing")

In [ ]:
#@title 🎤 Step 3: Upload Audio Files { display-mode: "form" }
#@markdown Upload clean speech and noise audio files for evaluation.

# Upload Audio Files
print("📤 Upload Clean Audio (required):")
uploaded_clean = files.upload()
audio_clean_file = list(uploaded_clean.keys())[0]
shutil.move(audio_clean_file, f'uploads/{audio_clean_file}')

print("\n📤 Upload Noise Audio (optional, for deterministic method):")
uploaded_noise = files.upload()
audio_noise_file = list(uploaded_noise.keys())[0] if uploaded_noise else None
if audio_noise_file:
    shutil.move(audio_noise_file, f'uploads/{audio_noise_file}')
    print(f"✓ Noise file uploaded")
else:
    print("✓ Skip - will use synthetic noise")

# Load audio
def load_audio(path, sr=16000):
    audio, orig_sr = sf.read(path)
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    if orig_sr != sr:
        num_samples = int(len(audio) * sr / orig_sr)
        audio = signal.resample(audio, num_samples)
    return audio, sr

audio_clean, sr = load_audio(f'uploads/{audio_clean_file}', SAMPLE_RATE)
audio_noise_uploaded = load_audio(f'uploads/{audio_noise_file}', SAMPLE_RATE)[0] if audio_noise_file else None

print(f"\n✅ Clean audio loaded: {len(audio_clean)/sr:.2f}s @ {sr}Hz")
if audio_noise_uploaded is not None:
    print(f"✅ Noise audio loaded: {len(audio_noise_uploaded)/sr:.2f}s")

In [ ]:
#@title ⚙️ Step 4: Load Processing Functions { display-mode: "form" }
#@markdown Initialize all noise generation, processing methods, and metrics calculation functions.

# ============== NOISE GENERATION & SNR CALCULATION ==============
def calculate_rms(signal):
    """Calculate Root Mean Square (RMS) power"""
    return np.sqrt(np.mean(signal ** 2))

def calculate_snr_db(clean, noisy):
    """Calculate actual SNR in dB using RMS power"""
    clean_power = calculate_rms(clean) ** 2
    noise_power = calculate_rms(noisy - clean) ** 2
    if noise_power < 1e-10:
        return float('inf')
    return 10 * np.log10(clean_power / noise_power)

def add_gaussian_noise(audio, target_snr_db):
    """Add Gaussian noise at specified SNR"""
    audio_power = calculate_rms(audio) ** 2
    if audio_power < 1e-10:
        audio_power = 1e-10
    noise_power = audio_power / (10 ** (target_snr_db / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), len(audio))
    mixed = audio + noise
    actual_snr = calculate_snr_db(audio, mixed)
    return mixed, noise, actual_snr

def add_white_noise(audio, target_snr_db):
    """Add white noise at specified SNR"""
    audio_power = calculate_rms(audio) ** 2
    if audio_power < 1e-10:
        audio_power = 1e-10
    noise_power = audio_power / (10 ** (target_snr_db / 10))
    noise = np.random.uniform(-1, 1, len(audio))
    noise = noise / calculate_rms(noise) * np.sqrt(noise_power)
    mixed = audio + noise
    actual_snr = calculate_snr_db(audio, mixed)
    return mixed, noise, actual_snr

def mix_audio_with_snr(clean, noise, target_snr_db):
    """Mix clean audio with noise at specified SNR"""
    if len(noise) < len(clean):
        repeats = int(np.ceil(len(clean) / len(noise)))
        noise = np.tile(noise, repeats)[:len(clean)]
    else:
        noise = noise[:len(clean)]
    
    clean_power = calculate_rms(clean) ** 2
    noise_power_original = calculate_rms(noise) ** 2
    
    if noise_power_original < 1e-10:
        noise_power_original = 1e-10
    if clean_power < 1e-10:
        clean_power = 1e-10
    
    target_noise_power = clean_power / (10 ** (target_snr_db / 10))
    scale = np.sqrt(target_noise_power / noise_power_original)
    scaled_noise = noise * scale
    mixed = clean + scaled_noise
    
    max_val = np.max(np.abs(mixed))
    if max_val > 1.0:
        mixed = mixed / max_val
        scaled_noise = scaled_noise / max_val
    
    actual_snr = calculate_snr_db(clean, mixed)
    return mixed, scaled_noise, actual_snr

# ============== PROCESSING METHODS ==============
def process_dtln(audio):
    """Process audio using DTLN ONNX models"""
    for inp in interpreter_1.get_inputs():
        model_inputs_1[inp.name] = np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32)
    for inp in interpreter_2.get_inputs():
        model_inputs_2[inp.name] = np.zeros([dim if isinstance(dim, int) else 1 for dim in inp.shape], dtype=np.float32)
    
    out_file = np.zeros(len(audio))
    in_buffer = np.zeros(BLOCK_LEN, dtype='float32')
    out_buffer = np.zeros(BLOCK_LEN, dtype='float32')
    num_blocks = (len(audio) - (BLOCK_LEN - BLOCK_SHIFT)) // BLOCK_SHIFT
    
    for idx in range(num_blocks):
        in_buffer[:-BLOCK_SHIFT] = in_buffer[BLOCK_SHIFT:]
        in_buffer[-BLOCK_SHIFT:] = audio[idx*BLOCK_SHIFT:(idx*BLOCK_SHIFT)+BLOCK_SHIFT]
        
        in_fft = np.fft.rfft(in_buffer)
        in_mag, in_phase = np.abs(in_fft), np.angle(in_fft)
        
        model_inputs_1[model_input_names_1[0]] = np.reshape(in_mag, (1,1,-1)).astype('float32')
        out_1 = interpreter_1.run(None, model_inputs_1)
        model_inputs_1[model_input_names_1[1]] = out_1[1]
        
        est_complex = in_mag * out_1[0] * np.exp(1j * in_phase)
        est_block = np.fft.irfft(est_complex)
        
        model_inputs_2[model_input_names_2[0]] = np.reshape(est_block, (1,1,-1)).astype('float32')
        out_2 = interpreter_2.run(None, model_inputs_2)
        model_inputs_2[model_input_names_2[1]] = out_2[1]
        
        out_buffer[:-BLOCK_SHIFT] = out_buffer[BLOCK_SHIFT:]
        out_buffer[-BLOCK_SHIFT:] = 0
        out_buffer += np.squeeze(out_2[0])
        out_file[idx*BLOCK_SHIFT:(idx*BLOCK_SHIFT)+BLOCK_SHIFT] = out_buffer[:BLOCK_SHIFT]
    return out_file

def spectral_subtraction_manual(noisy_audio, alpha=2.0, beta=0.01, sr=16000):
    """
    Spectral Subtraction based on Boll (1979)
    Reference: "Suppression of acoustic noise in speech using spectral subtraction"
    IEEE Transactions on Acoustics, Speech, and Signal Processing
    """
    nperseg = 512
    noverlap = 384
    
    f, t, Zxx = signal.stft(noisy_audio, fs=sr, nperseg=nperseg, noverlap=noverlap)
    
    # Estimate noise from initial frames (first 10 frames)
    noise_frames = 10
    noise_spectrum = np.mean(np.abs(Zxx[:, :noise_frames]) ** 2, axis=1, keepdims=True)
    
    # Spectral subtraction
    magnitude = np.abs(Zxx)
    phase = np.angle(Zxx)
    
    # Power spectral subtraction with over-subtraction factor (alpha) and spectral floor (beta)
    clean_magnitude_squared = np.maximum(
        magnitude ** 2 - alpha * noise_spectrum,
        beta * magnitude ** 2
    )
    clean_magnitude = np.sqrt(clean_magnitude_squared)
    
    clean_stft = clean_magnitude * np.exp(1j * phase)
    _, clean_audio = signal.istft(clean_stft, fs=sr, nperseg=nperseg, noverlap=noverlap)
    
    # Match length
    if len(clean_audio) > len(noisy_audio):
        clean_audio = clean_audio[:len(noisy_audio)]
    elif len(clean_audio) < len(noisy_audio):
        clean_audio = np.pad(clean_audio, (0, len(noisy_audio) - len(clean_audio)))
    
    return clean_audio

def wiener_filter_manual(noisy_audio, sr=16000):
    """
    Wiener Filter based on Wiener (1949) and Lim & Oppenheim (1979)
    Reference: "Enhancement and bandwidth compression of noisy speech"
    Proceedings of the IEEE
    """
    nperseg = 512
    noverlap = 384
    
    f, t, Zxx = signal.stft(noisy_audio, fs=sr, nperseg=nperseg, noverlap=noverlap)
    
    # Estimate noise power from initial frames
    noise_frames = 10
    noise_power = np.mean(np.abs(Zxx[:, :noise_frames]) ** 2, axis=1, keepdims=True)
    
    # Wiener filtering
    noisy_power = np.abs(Zxx) ** 2
    
    # Wiener gain with SNR estimation
    snr_prior = np.maximum(noisy_power - noise_power, 0) / (noise_power + 1e-10)
    wiener_gain = snr_prior / (snr_prior + 1)
    
    # Apply minimum gain threshold
    wiener_gain = np.maximum(wiener_gain, 0.1)
    
    clean_stft = Zxx * wiener_gain
    _, clean_audio = signal.istft(clean_stft, fs=sr, nperseg=nperseg, noverlap=noverlap)
    
    # Match length
    if len(clean_audio) > len(noisy_audio):
        clean_audio = clean_audio[:len(noisy_audio)]
    elif len(clean_audio) < len(noisy_audio):
        clean_audio = np.pad(clean_audio, (0, len(noisy_audio) - len(clean_audio)))
    
    return clean_audio

def process_traditional_library(noisy_audio, sr=16000):
    """
    Library-based noise reduction using noisereduce
    Algorithm: Spectral Gating (similar to Audacity)
    """
    reduced_audio = nr.reduce_noise(
        y=noisy_audio,
        sr=sr,
        stationary=True,
        prop_decrease=1.0
    )
    
    if len(reduced_audio) != len(noisy_audio):
        if len(reduced_audio) > len(noisy_audio):
            reduced_audio = reduced_audio[:len(noisy_audio)]
        else:
            reduced_audio = np.pad(reduced_audio, (0, len(noisy_audio) - len(reduced_audio)))
    
    return reduced_audio

# ============== METRICS CALCULATION ==============
def calculate_metrics(clean, processed, fs=16000):
    """Calculate evaluation metrics with time-alignment"""
    metrics = {}
    
    min_len = min(len(clean), len(processed))
    clean = clean[:min_len]
    processed = processed[:min_len]
    
    clean = np.nan_to_num(clean, nan=0.0, posinf=1.0, neginf=-1.0)
    processed = np.nan_to_num(processed, nan=0.0, posinf=1.0, neginf=-1.0)
    
    if np.sum(np.abs(clean)) == 0:
        clean = clean + 1e-10
    if np.sum(np.abs(processed)) == 0:
        processed = processed + 1e-10
    
    # Time-alignment
    from scipy.signal import correlate
    max_lag_search = min(512, len(processed) // 4)
    correlation = correlate(clean, processed, mode='full', method='fft')
    center = len(processed) - 1
    search_start = max(0, center - 50)
    search_end = min(len(correlation), center + max_lag_search)
    restricted_correlation = correlation[search_start:search_end]
    lag = np.argmax(restricted_correlation) + search_start - center
    
    if lag > 0:
        processed_aligned = processed[lag:]
        clean_aligned = clean[:len(processed_aligned)]
    elif lag < 0:
        clean_aligned = clean[-lag:]
        processed_aligned = processed[:len(clean_aligned)]
    else:
        clean_aligned = clean
        processed_aligned = processed
    
    min_len_aligned = min(len(clean_aligned), len(processed_aligned))
    clean_aligned = clean_aligned[:min_len_aligned]
    processed_aligned = processed_aligned[:min_len_aligned]
    
    # Calculate metrics
    try:
        metrics['stoi'] = stoi(clean_aligned, processed_aligned, fs, extended=False)
        if np.isnan(metrics['stoi']) or np.isinf(metrics['stoi']):
            metrics['stoi'] = 0.0
    except:
        metrics['stoi'] = 0.0
    
    try:
        if fs == 16000:
            metrics['pesq'] = pesq(fs, clean_aligned, processed_aligned, 'wb')
        else:
            metrics['pesq'] = 0.0
        if np.isnan(metrics['pesq']) or np.isinf(metrics['pesq']):
            metrics['pesq'] = 0.0
    except:
        metrics['pesq'] = 0.0
    
    metrics['mse'] = np.mean((clean_aligned - processed_aligned) ** 2)
    if np.isnan(metrics['mse']) or np.isinf(metrics['mse']):
        metrics['mse'] = 0.0
    
    amplitude_threshold = 0.01
    mask = np.abs(clean_aligned) > amplitude_threshold
    if np.sum(mask) > 0:
        clean_masked = clean_aligned[mask]
        processed_masked = processed_aligned[mask]
        epsilon = 1e-10
        relative_error = np.abs(clean_masked - processed_masked) / (np.abs(clean_masked) + epsilon)
        relative_error = np.clip(relative_error, 0, 10)
        metrics['mre'] = np.mean(relative_error)
    else:
        metrics['mre'] = 0.0
    
    if np.isnan(metrics['mre']) or np.isinf(metrics['mre']):
        metrics['mre'] = 0.0
    
    metrics['lag_samples'] = lag
    
    return metrics

print("✅ All processing functions loaded!")
print("\n📊 Available Methods:")
print("   1. Deterministic (DTLN)")
print("   2. Stochastic (DTLN)")
print("   3. Traditional-Manual (Spectral Subtraction & Wiener)")
print("   4. Traditional-Library (noisereduce)")
print("\n📐 SNR Range: -5, 0, 5, 10 dB")

---

## 🎯 Single Experiment Mode
**Test individual method with complete before/after analysis**

Run a single experiment to see detailed visualizations including:
- Waveforms (Clean, Noise, Before, After, Overlay)
- Spectrograms for all stages
- Reconstruction error analysis
- Metrics comparison chart
- Audio playback

In [ ]:
#@title 🧪 Run Single Experiment { display-mode: "form" }

#@markdown ---
#@markdown ### 🔧 **Configuration**

run_single = False  #@param {type:"boolean"}

#@markdown ---
#@markdown ### 🎯 **Method Selection**
single_method = "deterministic"  #@param ["deterministic", "stochastic", "traditional_manual", "traditional_library"]

#@markdown ---
#@markdown ### 🛠️ **DSP Algorithm** (for Traditional-Manual only)
single_dsp_method = "spectral_subtraction"  #@param ["spectral_subtraction", "wiener"]

#@markdown ---
#@markdown ### 🔊 **Noise Type** (for Stochastic/Traditional)
single_noise_type = "gaussian"  #@param ["gaussian", "white", "mixed", "uploaded"]

#@markdown ---
#@markdown ### 📊 **SNR Level** (dB)
single_snr = 10  #@param {type:"slider", min:-5, max:10, step:5}

if run_single:
    print("="*100)
    print("🧪 SINGLE EXPERIMENT - FULL ANALYSIS")
    print("="*100)
    
    # Generate noisy audio based on method
    if single_method == 'deterministic':
        if audio_noise_uploaded is None:
            print("❌ ERROR: Deterministic requires uploaded noise file!")
            raise ValueError("Upload noise file first")
        mixed_audio, used_noise, actual_snr = mix_audio_with_snr(audio_clean, audio_noise_uploaded, single_snr)
        method_label = "Deterministic (DTLN)"
        noise_label = "Uploaded"
        
    elif single_method == 'stochastic':
        if single_noise_type == 'gaussian':
            mixed_audio, used_noise, actual_snr = add_gaussian_noise(audio_clean, single_snr)
            noise_label = "Gaussian"
        elif single_noise_type == 'white':
            mixed_audio, used_noise, actual_snr = add_white_noise(audio_clean, single_snr)
            noise_label = "White"
        elif single_noise_type == 'mixed':
            if audio_noise_uploaded is not None:
                noise_gaussian = add_gaussian_noise(audio_clean, single_snr)[1]
                noise_white = add_white_noise(audio_clean, single_snr)[1]
                noise_uploaded_scaled = mix_audio_with_snr(audio_clean, audio_noise_uploaded, single_snr)[1]
                used_noise = (noise_gaussian + noise_white + noise_uploaded_scaled) / 3.0
            else:
                noise_gaussian = add_gaussian_noise(audio_clean, single_snr)[1]
                noise_white = add_white_noise(audio_clean, single_snr)[1]
                used_noise = (noise_gaussian + noise_white) / 2.0
            mixed_audio = audio_clean + used_noise
            actual_snr = calculate_snr_db(audio_clean, mixed_audio)
            noise_label = "Mixed"
        elif single_noise_type == 'uploaded':
            if audio_noise_uploaded is None:
                print("⚠️  No uploaded noise, using Gaussian")
                mixed_audio, used_noise, actual_snr = add_gaussian_noise(audio_clean, single_snr)
                noise_label = "Gaussian"
            else:
                mixed_audio, used_noise, actual_snr = mix_audio_with_snr(audio_clean, audio_noise_uploaded, single_snr)
                noise_label = "Uploaded"
        method_label = "Stochastic (DTLN)"
        
    elif single_method == 'traditional_manual':
        if single_noise_type == 'gaussian':
            mixed_audio, used_noise, actual_snr = add_gaussian_noise(audio_clean, single_snr)
            noise_label = "Gaussian"
        elif single_noise_type == 'white':
            mixed_audio, used_noise, actual_snr = add_white_noise(audio_clean, single_snr)
            noise_label = "White"
        elif single_noise_type == 'mixed':
            if audio_noise_uploaded is not None:
                noise_gaussian = add_gaussian_noise(audio_clean, single_snr)[1]
                noise_white = add_white_noise(audio_clean, single_snr)[1]
                noise_uploaded_scaled = mix_audio_with_snr(audio_clean, audio_noise_uploaded, single_snr)[1]
                used_noise = (noise_gaussian + noise_white + noise_uploaded_scaled) / 3.0
            else:
                noise_gaussian = add_gaussian_noise(audio_clean, single_snr)[1]
                noise_white = add_white_noise(audio_clean, single_snr)[1]
                used_noise = (noise_gaussian + noise_white) / 2.0
            mixed_audio = audio_clean + used_noise
            actual_snr = calculate_snr_db(audio_clean, mixed_audio)
            noise_label = "Mixed"
        elif single_noise_type == 'uploaded':
            if audio_noise_uploaded is None:
                print("⚠️  No uploaded noise, using Gaussian")
                mixed_audio, used_noise, actual_snr = add_gaussian_noise(audio_clean, single_snr)
                noise_label = "Gaussian"
            else:
                mixed_audio, used_noise, actual_snr = mix_audio_with_snr(audio_clean, audio_noise_uploaded, single_snr)
                noise_label = "Uploaded"
        method_label = f"Traditional-Manual ({single_dsp_method.replace('_', ' ').title()})"
        
    else:  # traditional_library
        if single_noise_type == 'gaussian':
            mixed_audio, used_noise, actual_snr = add_gaussian_noise(audio_clean, single_snr)
            noise_label = "Gaussian"
        elif single_noise_type == 'white':
            mixed_audio, used_noise, actual_snr = add_white_noise(audio_clean, single_snr)
            noise_label = "White"
        elif single_noise_type == 'mixed':
            if audio_noise_uploaded is not None:
                noise_gaussian = add_gaussian_noise(audio_clean, single_snr)[1]
                noise_white = add_white_noise(audio_clean, single_snr)[1]
                noise_uploaded_scaled = mix_audio_with_snr(audio_clean, audio_noise_uploaded, single_snr)[1]
                used_noise = (noise_gaussian + noise_white + noise_uploaded_scaled) / 3.0
            else:
                noise_gaussian = add_gaussian_noise(audio_clean, single_snr)[1]
                noise_white = add_white_noise(audio_clean, single_snr)[1]
                used_noise = (noise_gaussian + noise_white) / 2.0
            mixed_audio = audio_clean + used_noise
            actual_snr = calculate_snr_db(audio_clean, mixed_audio)
            noise_label = "Mixed"
        elif single_noise_type == 'uploaded':
            if audio_noise_uploaded is None:
                print("⚠️  No uploaded noise, using Gaussian")
                mixed_audio, used_noise, actual_snr = add_gaussian_noise(audio_clean, single_snr)
                noise_label = "Gaussian"
            else:
                mixed_audio, used_noise, actual_snr = mix_audio_with_snr(audio_clean, audio_noise_uploaded, single_snr)
                noise_label = "Uploaded"
        method_label = "Traditional-Library (noisereduce)"
    
    print(f"Method: {method_label}")
    print(f"Noise: {noise_label}")
    print(f"Target SNR: {single_snr} dB | Actual SNR: {actual_snr:.2f} dB")
    print()
    
    # Calculate baseline
    print("⚙️  Calculating baseline metrics...")
    baseline = calculate_metrics(audio_clean, mixed_audio, SAMPLE_RATE)
    
    # Process audio
    print("⚙️  Processing audio...")
    start = time.time()
    if single_method in ['deterministic', 'stochastic']:
        audio_processed = process_dtln(mixed_audio)
    elif single_method == 'traditional_manual':
        if single_dsp_method == 'spectral_subtraction':
            audio_processed = spectral_subtraction_manual(mixed_audio, sr=SAMPLE_RATE)
        else:
            audio_processed = wiener_filter_manual(mixed_audio, sr=SAMPLE_RATE)
    else:  # traditional_library
        audio_processed = process_traditional_library(mixed_audio, sr=SAMPLE_RATE)
    proc_time = time.time() - start
    
    # Calculate processed metrics
    metrics = calculate_metrics(audio_clean, audio_processed, SAMPLE_RATE)
    
    # Helper function to get status and color
    def get_metric_status(metric_name, value):
        if metric_name == 'STOI':
            # Range: 0-1 (Higher is better)
            if value >= 0.9: return '🟢 Excellent', '#10b981'
            elif value >= 0.7: return '🟡 Good', '#f59e0b'
            elif value >= 0.5: return '🟠 Fair', '#fb923c'
            else: return '🔴 Poor', '#ef4444'
        elif metric_name == 'PESQ':
            # Range: -0.5 to 4.5 (Higher is better)
            if value >= 3.5: return '🟢 Excellent', '#10b981'
            elif value >= 2.5: return '🟡 Good', '#f59e0b'
            elif value >= 1.5: return '🟠 Fair', '#fb923c'
            else: return '🔴 Poor', '#ef4444'
        elif metric_name == 'MSE':
            # Range: 0-∞ (Lower is better)
            if value <= 0.001: return '🟢 Excellent', '#10b981'
            elif value <= 0.01: return '🟡 Good', '#f59e0b'
            elif value <= 0.1: return '🟠 Fair', '#fb923c'
            else: return '🔴 Poor', '#ef4444'
        elif metric_name == 'MRE':
            # Range: 0-10 (Lower is better)
            if value <= 0.1: return '🟢 Excellent', '#10b981'
            elif value <= 0.5: return '🟡 Good', '#f59e0b'
            elif value <= 1.0: return '🟠 Fair', '#fb923c'
            else: return '🔴 Poor', '#ef4444'
    
    # Display metrics with status
    print(f"\n{'='*110}")
    print(f"📊 EVALUATION RESULTS")
    print(f"{'='*110}")
    print(f"Processing Time: {proc_time:.3f}s")
    print()
    print(f"{'Metric':<12} {'Range':<20} {'Baseline':<12} {'Status':<20} {'Processed':<12} {'Status':<20}")
    print(f"{'-'*110}")
    
    stoi_base_status, _ = get_metric_status('STOI', baseline['stoi'])
    stoi_proc_status, _ = get_metric_status('STOI', metrics['stoi'])
    print(f"{'STOI':<12} {'0-1 (↑ better)':<20} {baseline['stoi']:<12.4f} {stoi_base_status:<20} {metrics['stoi']:<12.4f} {stoi_proc_status:<20}")
    
    pesq_base_status, _ = get_metric_status('PESQ', baseline['pesq'])
    pesq_proc_status, _ = get_metric_status('PESQ', metrics['pesq'])
    print(f"{'PESQ':<12} {'-0.5-4.5 (↑ better)':<20} {baseline['pesq']:<12.4f} {pesq_base_status:<20} {metrics['pesq']:<12.4f} {pesq_proc_status:<20}")
    
    mse_base_status, _ = get_metric_status('MSE', baseline['mse'])
    mse_proc_status, _ = get_metric_status('MSE', metrics['mse'])
    print(f"{'MSE':<12} {'0-∞ (↓ better)':<20} {baseline['mse']:<12.4f} {mse_base_status:<20} {metrics['mse']:<12.4f} {mse_proc_status:<20}")
    
    mre_base_status, _ = get_metric_status('MRE', baseline['mre'])
    mre_proc_status, _ = get_metric_status('MRE', metrics['mre'])
    print(f"{'MRE':<12} {'0-10 (↓ better)':<20} {baseline['mre']:<12.4f} {mre_base_status:<20} {metrics['mre']:<12.4f} {mre_proc_status:<20}")
    print(f"{'='*110}")
    
    # Audio players
    print("\n🎵 Audio Playback:")
    print("Clean Audio:")
    display(Audio(audio_clean, rate=SAMPLE_RATE))
    print("\nNoisy Audio (Before):")
    display(Audio(mixed_audio, rate=SAMPLE_RATE))
    print("\nProcessed Audio (After):")
    display(Audio(audio_processed, rate=SAMPLE_RATE))
    
    # ============== FULL VISUALIZATION ==============
    time_axis = np.linspace(0, len(audio_clean)/SAMPLE_RATE, len(audio_clean))
    
    # Figure 1: Waveforms + Spectrograms (5 rows)
    fig = plt.figure(figsize=(20, 18))
    gs = fig.add_gridspec(5, 4, hspace=0.4, wspace=0.3)
    fig.suptitle(f'{method_label} | {noise_label} Noise | SNR={actual_snr:.1f}dB', 
                 fontsize=18, fontweight='bold', y=0.995)
    
    # Row 1: Clean
    ax = fig.add_subplot(gs[0, :2])
    ax.plot(time_axis, audio_clean, linewidth=0.5, color='#10b981', alpha=0.9)
    ax.set_title('Clean Audio - Waveform', fontweight='bold', fontsize=12)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True, alpha=0.3)
    ax.set_ylim([-1, 1])
    
    ax = fig.add_subplot(gs[0, 2:])
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_clean)), ref=np.max)
    img = librosa.display.specshow(D, sr=SAMPLE_RATE, x_axis='time', y_axis='hz', ax=ax, cmap='magma')
    ax.set_title('Clean Audio - Spectrogram', fontweight='bold', fontsize=12)
    plt.colorbar(img, ax=ax, format='%+2.0f dB')
    
    # Row 2: Noise
    ax = fig.add_subplot(gs[1, :2])
    ax.plot(time_axis[:len(used_noise)], used_noise, linewidth=0.5, color='#f59e0b', alpha=0.9)
    ax.set_title('Noise - Waveform', fontweight='bold', fontsize=12)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True, alpha=0.3)
    ax.set_ylim([-1, 1])
    
    ax = fig.add_subplot(gs[1, 2:])
    D = librosa.amplitude_to_db(np.abs(librosa.stft(used_noise)), ref=np.max)
    img = librosa.display.specshow(D, sr=SAMPLE_RATE, x_axis='time', y_axis='hz', ax=ax, cmap='magma')
    ax.set_title('Noise - Spectrogram', fontweight='bold', fontsize=12)
    plt.colorbar(img, ax=ax, format='%+2.0f dB')
    
    # Row 3: Noisy (Before)
    ax = fig.add_subplot(gs[2, :2])
    ax.plot(time_axis, mixed_audio, linewidth=0.5, color='#ef4444', alpha=0.9)
    ax.set_title('Noisy Audio (BEFORE) - Waveform', fontweight='bold', fontsize=12)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True, alpha=0.3)
    ax.set_ylim([-1, 1])
    
    ax = fig.add_subplot(gs[2, 2:])
    D = librosa.amplitude_to_db(np.abs(librosa.stft(mixed_audio)), ref=np.max)
    img = librosa.display.specshow(D, sr=SAMPLE_RATE, x_axis='time', y_axis='hz', ax=ax, cmap='magma')
    ax.set_title('Noisy Audio (BEFORE) - Spectrogram', fontweight='bold', fontsize=12)
    plt.colorbar(img, ax=ax, format='%+2.0f dB')
    
    # Row 4: Processed (After)
    ax = fig.add_subplot(gs[3, :2])
    ax.plot(time_axis, audio_processed, linewidth=0.5, color='#3b82f6', alpha=0.9)
    ax.set_title('Processed Audio (AFTER) - Waveform', fontweight='bold', fontsize=12)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.grid(True, alpha=0.3)
    ax.set_ylim([-1, 1])
    
    ax = fig.add_subplot(gs[3, 2:])
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_processed)), ref=np.max)
    img = librosa.display.specshow(D, sr=SAMPLE_RATE, x_axis='time', y_axis='hz', ax=ax, cmap='magma')
    ax.set_title('Processed Audio (AFTER) - Spectrogram', fontweight='bold', fontsize=12)
    plt.colorbar(img, ax=ax, format='%+2.0f dB')
    
    # Row 5: Overlay Before/After + Error
    ax = fig.add_subplot(gs[4, :2])
    ax.plot(time_axis, mixed_audio, linewidth=0.8, color='#ef4444', alpha=0.6, label='BEFORE (Noisy)')
    ax.plot(time_axis, audio_processed, linewidth=0.8, color='#3b82f6', alpha=0.8, label='AFTER (Processed)')
    ax.plot(time_axis, audio_clean, linewidth=0.5, color='#10b981', alpha=0.4, label='Reference (Clean)', linestyle='--')
    ax.set_title('🔍 OVERLAY: Before vs After', fontweight='bold', fontsize=13)
    ax.set_xlabel('Time (s)', fontweight='bold')
    ax.set_ylabel('Amplitude', fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_ylim([-1, 1])
    ax.legend(fontsize=10, loc='upper right')
    ax.set_facecolor('#f8f9fa')
    
    ax = fig.add_subplot(gs[4, 2:])
    difference = audio_clean - audio_processed
    ax.plot(time_axis, difference, linewidth=0.6, color='#8b5cf6', alpha=0.8)
    ax.fill_between(time_axis, difference, 0, alpha=0.3, color='#8b5cf6')
    ax.set_title('📉 Reconstruction Error', fontweight='bold', fontsize=13)
    ax.set_xlabel('Time (s)', fontweight='bold')
    ax.set_ylabel('Error', fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
    ax.set_facecolor('#f8f9fa')
    
    error_mean = np.mean(np.abs(difference))
    error_std = np.std(difference)
    error_max = np.max(np.abs(difference))
    ax.text(0.02, 0.98, f'Mean: {error_mean:.4f}\nStd: {error_std:.4f}\nMax: {error_max:.4f}',
            transform=ax.transAxes, fontsize=9, va='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    fname = f"single_{single_method}_{noise_label}_{single_snr}dB"
    plt.savefig(f'results/spectrograms/{fname}_full.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Figure 2: Metrics Bar Chart with Status Colors
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    fig.suptitle(f'Metrics Scoring: {method_label}', fontsize=14, fontweight='bold')
    
    metrics_data = [
        ('STOI\n(0-1, ↑ better)', baseline['stoi'], metrics['stoi'], [0, 1], 'STOI'),
        ('PESQ\n(-0.5-4.5, ↑ better)', baseline['pesq'], metrics['pesq'], [-0.5, 4.5], 'PESQ'),
        ('MSE\n(0-∞, ↓ better)', baseline['mse'], metrics['mse'], None, 'MSE'),
        ('MRE\n(0-10, ↓ better)', baseline['mre'], metrics['mre'], [0, 10], 'MRE')
    ]
    
    for idx, (title, base_val, proc_val, ylim, metric_name) in enumerate(metrics_data):
        ax = axes[idx]
        
        # Get colors based on status
        _, base_color = get_metric_status(metric_name, base_val)
        _, proc_color = get_metric_status(metric_name, proc_val)
        
        bar1 = ax.bar(['Baseline'], [base_val], color=base_color, alpha=0.6, width=0.6, edgecolor='black', linewidth=1.5)
        bar2 = ax.bar(['Processed'], [proc_val], color=proc_color, alpha=0.9, width=0.6, edgecolor='black', linewidth=1.5)
        ax.set_title(title, fontweight='bold', fontsize=10)
        ax.grid(axis='y', alpha=0.3)
        if ylim:
            ax.set_ylim(ylim)
        
        # Add value labels
        for bar in [bar1, bar2]:
            for b in bar:
                height = b.get_height()
                ax.text(b.get_x() + b.get_width()/2., height,
                       f'{height:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
        
        # Add status legend
        base_status, _ = get_metric_status(metric_name, base_val)
        proc_status, _ = get_metric_status(metric_name, proc_val)
        ax.text(0.5, 0.95, f'Baseline: {base_status.split()[0]}\nProcessed: {proc_status.split()[0]}',
                transform=ax.transAxes, ha='center', va='top', fontsize=8,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(f'results/metrics/{fname}_chart.png', dpi=120, bbox_inches='tight')
    plt.show()
    
    # Save audio files
    sf.write(f'outputs/{fname}_before.wav', mixed_audio, SAMPLE_RATE)
    sf.write(f'outputs/{fname}_after.wav', audio_processed, SAMPLE_RATE)
    
    print(f"\n✅ Single experiment completed!")
    print(f"   Files saved:")
    print(f"   • results/spectrograms/{fname}_full.png")
    print(f"   • results/metrics/{fname}_chart.png")
    print(f"   • outputs/{fname}_before.wav")
    print(f"   • outputs/{fname}_after.wav")

---

## 📊 Batch Evaluation Mode
**Comprehensive automated testing with two independent scenarios**

### **🌍 Scenario 1: Real-world Comparison**
- Uses uploaded noise file
- Tests: Deterministic (DTLN) + Traditional-Manual (SS & Wiener) + Traditional-Library
- **Total: 16 experiments** (4 methods × 4 SNR levels)

### **🔬 Scenario 2: Synthetic Comparison**
- Uses generated noise (Gaussian, White, Mixed)
- Tests: Stochastic (DTLN) + Traditional-Manual (SS & Wiener) + Traditional-Library
- **Total: 48 experiments** (4 methods × 3 noise types × 4 SNR levels)

**Grand Total: 64 experiments**

In [ ]:
#@title 🚀 Run Batch Evaluation (Two Scenarios) { display-mode: "form" }

#@markdown ---
#@markdown ### ⚙️ **Execution Control**
run_batch = False  #@param {type:"boolean"}

#@markdown ---
#@markdown ### 📝 **Note:**
#@markdown - Scenario 1 requires uploaded noise file
#@markdown - Scenario 2 uses synthetic noise (always available)
#@markdown - Results exported to separate Excel files
#@markdown - Visualizations saved automatically

if run_batch:
    SNR_LEVELS = [-5, 0, 5, 10]
    
    # ============================================================================
    # SCENARIO 1: REAL-WORLD COMPARISON (Uploaded Noise Only)
    # ============================================================================
    print("="*100)
    print("🌍 SCENARIO 1: REAL-WORLD COMPARISON (Using Uploaded Noise)")
    print("="*100)
    print("Comparing: Deterministic (DTLN) vs Traditional-Manual (Both DSP) vs Traditional-Library")
    print("Noise Source: Uploaded noise file (same across all methods)")
    print(f"SNR Levels: {SNR_LEVELS}")
    print()
    
    realworld_results = []
    
    if audio_noise_uploaded is not None:
        scenario1_configs = []
        
        # Deterministic (DTLN)
        for snr in SNR_LEVELS:
            scenario1_configs.append({
                'method': 'deterministic',
                'snr': snr,
                'noise_type': 'uploaded'
            })
        
        # Traditional-Manual: BOTH Spectral Subtraction AND Wiener Filter
        for dsp_algo in ['spectral_subtraction', 'wiener']:
            for snr in SNR_LEVELS:
                scenario1_configs.append({
                    'method': 'traditional_manual',
                    'snr': snr,
                    'noise_type': 'uploaded',
                    'dsp_method': dsp_algo
                })
        
        # Traditional-Library
        for snr in SNR_LEVELS:
            scenario1_configs.append({
                'method': 'traditional_library',
                'snr': snr,
                'noise_type': 'uploaded'
            })
        
        total_s1 = len(scenario1_configs)
        print(f"🔄 Running {total_s1} experiments...")
        print(f"   - Deterministic (DTLN): {len(SNR_LEVELS)} configs")
        print(f"   - Traditional-Manual (Spectral Subtraction): {len(SNR_LEVELS)} configs")
        print(f"   - Traditional-Manual (Wiener Filter): {len(SNR_LEVELS)} configs")
        print(f"   - Traditional-Library (noisereduce): {len(SNR_LEVELS)} configs")
        print()
        
        for idx, config in enumerate(scenario1_configs, 1):
            method = config['method']
            snr_db = config['snr']
            
            progress = f"[{idx}/{total_s1}]"
            
            try:
                mixed_audio, used_noise, actual_snr = mix_audio_with_snr(audio_clean, audio_noise_uploaded, snr_db)
                
                if method == 'deterministic':
                    method_label = "Deterministic (DTLN)"
                elif method == 'traditional_manual':
                    method_label = f"Traditional-Manual ({config['dsp_method'].replace('_', ' ').title()})"
                else:
                    method_label = "Traditional-Library (noisereduce)"
                
                print(f"{progress} {method_label:<50} | SNR={actual_snr:6.2f}dB", end='')
                
                baseline = calculate_metrics(audio_clean, mixed_audio, SAMPLE_RATE)
                
                start = time.time()
                if method == 'deterministic':
                    audio_proc = process_dtln(mixed_audio)
                elif method == 'traditional_manual':
                    if config['dsp_method'] == 'spectral_subtraction':
                        audio_proc = spectral_subtraction_manual(mixed_audio, sr=SAMPLE_RATE)
                    else:
                        audio_proc = wiener_filter_manual(mixed_audio, sr=SAMPLE_RATE)
                else:
                    audio_proc = process_traditional_library(mixed_audio, sr=SAMPLE_RATE)
                proc_time = time.time() - start
                
                m = calculate_metrics(audio_clean, audio_proc, SAMPLE_RATE)
                
                stoi_improvement = m['stoi'] - baseline['stoi']
                pesq_improvement = m['pesq'] - baseline['pesq']
                
                print(f" | STOI: {baseline['stoi']:.3f}→{m['stoi']:.3f} (Δ{stoi_improvement:+.3f}) | PESQ: {baseline['pesq']:.2f}→{m['pesq']:.2f} (Δ{pesq_improvement:+.2f})")
                
                realworld_results.append({
                    'Method': method_label,
                    'Noise_Source': 'Uploaded',
                    'Target_SNR_dB': snr_db,
                    'Actual_SNR_dB': actual_snr,
                    'Baseline_STOI': baseline['stoi'],
                    'Baseline_PESQ': baseline['pesq'],
                    'Processed_STOI': m['stoi'],
                    'Processed_PESQ': m['pesq'],
                    'STOI_Improvement': stoi_improvement,
                    'PESQ_Improvement': pesq_improvement,
                    'Baseline_MSE': baseline['mse'],
                    'Processed_MSE': m['mse'],
                    'MSE_Improvement': baseline['mse'] - m['mse'],
                    'Baseline_MRE': baseline['mre'],
                    'Processed_MRE': m['mre'],
                    'MRE_Improvement': baseline['mre'] - m['mre'],
                    'Processing_Time_s': proc_time,
                })
                
                fname = f"realworld_{method}_{config.get('dsp_method', 'dtln')}_{snr_db:.0f}dB"
                sf.write(f'results/audio/{fname}_processed.wav', audio_proc, SAMPLE_RATE)
                
            except Exception as e:
                print(f" ❌ Error: {str(e)}")
                continue
        
        print(f"\n✅ Scenario 1 completed: {len(realworld_results)} results")
    else:
        print("⚠️  SKIPPED: No uploaded noise file provided")
    
    # ============================================================================
    # SCENARIO 2: SYNTHETIC COMPARISON (Generated Noise)
    # ============================================================================
    print(f"\n{'='*100}")
    print("🔬 SCENARIO 2: SYNTHETIC COMPARISON (Using Generated Noise)")
    print("="*100)
    print("Comparing: Stochastic (DTLN) vs Traditional-Manual (Both DSP) vs Traditional-Library")
    print("Noise Sources: Gaussian, White, Mixed")
    print(f"SNR Levels: {SNR_LEVELS}")
    print()
    
    synthetic_results = []
    scenario2_configs = []
    noise_types = ['gaussian', 'white', 'mixed']
    
    # Stochastic (DTLN)
    for noise_type in noise_types:
        for snr in SNR_LEVELS:
            scenario2_configs.append({
                'method': 'stochastic',
                'snr': snr,
                'noise_type': noise_type
            })
    
    # Traditional-Manual: BOTH Spectral Subtraction AND Wiener Filter
    for dsp_algo in ['spectral_subtraction', 'wiener']:
        for noise_type in noise_types:
            for snr in SNR_LEVELS:
                scenario2_configs.append({
                    'method': 'traditional_manual',
                    'snr': snr,
                    'noise_type': noise_type,
                    'dsp_method': dsp_algo
                })
    
    # Traditional-Library
    for noise_type in noise_types:
        for snr in SNR_LEVELS:
            scenario2_configs.append({
                'method': 'traditional_library',
                'snr': snr,
                'noise_type': noise_type
            })
    
    total_s2 = len(scenario2_configs)
    print(f"🔄 Running {total_s2} experiments...")
    print(f"   - Stochastic (DTLN): {len(noise_types) * len(SNR_LEVELS)} configs")
    print(f"   - Traditional-Manual (Spectral Subtraction): {len(noise_types) * len(SNR_LEVELS)} configs")
    print(f"   - Traditional-Manual (Wiener Filter): {len(noise_types) * len(SNR_LEVELS)} configs")
    print(f"   - Traditional-Library (noisereduce): {len(noise_types) * len(SNR_LEVELS)} configs")
    print()
    
    for idx, config in enumerate(scenario2_configs, 1):
        method = config['method']
        snr_db = config['snr']
        noise_type = config['noise_type']
        
        progress = f"[{idx}/{total_s2}]"
        
        try:
            # Generate noisy audio
            if noise_type == 'gaussian':
                mixed_audio, used_noise, actual_snr = add_gaussian_noise(audio_clean, snr_db)
                noise_label = "Gaussian"
            elif noise_type == 'white':
                mixed_audio, used_noise, actual_snr = add_white_noise(audio_clean, snr_db)
                noise_label = "White"
            elif noise_type == 'mixed':
                if audio_noise_uploaded is not None:
                    noise_gaussian = add_gaussian_noise(audio_clean, snr_db)[1]
                    noise_white = add_white_noise(audio_clean, snr_db)[1]
                    noise_uploaded_scaled = mix_audio_with_snr(audio_clean, audio_noise_uploaded, snr_db)[1]
                    used_noise = (noise_gaussian + noise_white + noise_uploaded_scaled) / 3.0
                else:
                    noise_gaussian = add_gaussian_noise(audio_clean, snr_db)[1]
                    noise_white = add_white_noise(audio_clean, snr_db)[1]
                    used_noise = (noise_gaussian + noise_white) / 2.0
                mixed_audio = audio_clean + used_noise
                actual_snr = calculate_snr_db(audio_clean, mixed_audio)
                noise_label = "Mixed"
            
            if method == 'stochastic':
                method_label = "Stochastic (DTLN)"
            elif method == 'traditional_manual':
                method_label = f"Traditional-Manual ({config['dsp_method'].replace('_', ' ').title()})"
            else:
                method_label = "Traditional-Library (noisereduce)"
            
            print(f"{progress} {method_label:<50} | {noise_label:<10} | SNR={actual_snr:6.2f}dB", end='')
            
            baseline = calculate_metrics(audio_clean, mixed_audio, SAMPLE_RATE)
            
            start = time.time()
            if method == 'stochastic':
                audio_proc = process_dtln(mixed_audio)
            elif method == 'traditional_manual':
                if config['dsp_method'] == 'spectral_subtraction':
                    audio_proc = spectral_subtraction_manual(mixed_audio, sr=SAMPLE_RATE)
                else:
                    audio_proc = wiener_filter_manual(mixed_audio, sr=SAMPLE_RATE)
            else:
                audio_proc = process_traditional_library(mixed_audio, sr=SAMPLE_RATE)
            proc_time = time.time() - start
            
            m = calculate_metrics(audio_clean, audio_proc, SAMPLE_RATE)
            
            stoi_improvement = m['stoi'] - baseline['stoi']
            pesq_improvement = m['pesq'] - baseline['pesq']
            
            print(f" | STOI: {baseline['stoi']:.3f}→{m['stoi']:.3f} (Δ{stoi_improvement:+.3f})")
            
            synthetic_results.append({
                'Method': method_label,
                'Noise_Type': noise_label,
                'Target_SNR_dB': snr_db,
                'Actual_SNR_dB': actual_snr,
                'Baseline_STOI': baseline['stoi'],
                'Baseline_PESQ': baseline['pesq'],
                'Processed_STOI': m['stoi'],
                'Processed_PESQ': m['pesq'],
                'STOI_Improvement': stoi_improvement,
                'PESQ_Improvement': pesq_improvement,
                'Baseline_MSE': baseline['mse'],
                'Processed_MSE': m['mse'],
                'MSE_Improvement': baseline['mse'] - m['mse'],
                'Baseline_MRE': baseline['mre'],
                'Processed_MRE': m['mre'],
                'MRE_Improvement': baseline['mre'] - m['mre'],
                'Processing_Time_s': proc_time,
            })
            
            fname = f"synthetic_{method}_{config.get('dsp_method', 'dtln')}_{noise_label}_{snr_db:.0f}dB"
            sf.write(f'results/audio/{fname}_processed.wav', audio_proc, SAMPLE_RATE)
            
        except Exception as e:
            print(f" ❌ Error: {str(e)}")
            continue
    
    print(f"\n✅ Scenario 2 completed: {len(synthetic_results)} results")
    
    # ============================================================================
    # EXPORT RESULTS TO SEPARATE EXCEL FILES
    # ============================================================================
    print(f"\n{'='*100}")
    print("📊 EXPORTING RESULTS")
    print("="*100)
    
    # Scenario 1: Real-world
    if len(realworld_results) > 0:
        df_rw = pd.DataFrame(realworld_results)
        numeric_cols = df_rw.select_dtypes(include=[np.number]).columns
        df_rw[numeric_cols] = df_rw[numeric_cols].round(4)
        
        excel_rw = 'results/scenario1_realworld_comparison.xlsx'
        with pd.ExcelWriter(excel_rw, engine='openpyxl') as writer:
            df_rw.to_excel(writer, sheet_name='All_Results', index=False)
            
            summary_method = df_rw.groupby('Method')[['Baseline_STOI', 'Processed_STOI',
                                                       'Baseline_PESQ', 'Processed_PESQ',
                                                       'Baseline_MSE', 'Processed_MSE',
                                                       'Baseline_MRE', 'Processed_MRE']].agg(['mean', 'std']).round(4)
            summary_method.to_excel(writer, sheet_name='Summary_By_Method')
            
            summary_snr = df_rw.groupby('Actual_SNR_dB')[['Baseline_STOI', 'Processed_STOI',
                                                           'Baseline_PESQ', 'Processed_PESQ',
                                                           'Baseline_MSE', 'Processed_MSE',
                                                           'Baseline_MRE', 'Processed_MRE']].mean().round(4)
            summary_snr.to_excel(writer, sheet_name='Summary_By_SNR')
        
        print(f"✅ Scenario 1 exported: {excel_rw}")
    
    # Scenario 2: Synthetic
    if len(synthetic_results) > 0:
        df_syn = pd.DataFrame(synthetic_results)
        numeric_cols = df_syn.select_dtypes(include=[np.number]).columns
        df_syn[numeric_cols] = df_syn[numeric_cols].round(4)
        
        excel_syn = 'results/scenario2_synthetic_comparison.xlsx'
        with pd.ExcelWriter(excel_syn, engine='openpyxl') as writer:
            df_syn.to_excel(writer, sheet_name='All_Results', index=False)
            
            summary_method = df_syn.groupby('Method')[['Baseline_STOI', 'Processed_STOI',
                                                        'Baseline_PESQ', 'Processed_PESQ',
                                                        'Baseline_MSE', 'Processed_MSE',
                                                        'Baseline_MRE', 'Processed_MRE']].agg(['mean', 'std']).round(4)
            summary_method.to_excel(writer, sheet_name='Summary_By_Method')
            
            summary_noise = df_syn.groupby('Noise_Type')[['Baseline_STOI', 'Processed_STOI',
                                                           'Baseline_PESQ', 'Processed_PESQ',
                                                           'Baseline_MSE', 'Processed_MSE',
                                                           'Baseline_MRE', 'Processed_MRE']].mean().round(4)
            summary_noise.to_excel(writer, sheet_name='Summary_By_Noise')
            
            summary_snr = df_syn.groupby('Actual_SNR_dB')[['Baseline_STOI', 'Processed_STOI',
                                                            'Baseline_PESQ', 'Processed_PESQ',
                                                            'Baseline_MSE', 'Processed_MSE',
                                                            'Baseline_MRE', 'Processed_MRE']].mean().round(4)
            summary_snr.to_excel(writer, sheet_name='Summary_By_SNR')
        
        print(f"✅ Scenario 2 exported: {excel_syn}")
    
    # ============================================================================
    # VISUALIZATIONS: SCENARIO 1 - REAL-WORLD
    # ============================================================================
    if len(realworld_results) > 0:
        print(f"\n{'='*100}")
        print("📈 SCENARIO 1: REAL-WORLD VISUALIZATIONS")
        print("="*100)
        
        # Display scoring table
        print(f"\n{'Method':<50} {'SNR':<7} {'Baseline Scores (STOI/PESQ/MSE/MRE)':<50} {'Processed Scores (STOI/PESQ/MSE/MRE)':<50}")
        print("="*165)
        for r in realworld_results:
            base_scores = f"{r['Baseline_STOI']:.3f} / {r['Baseline_PESQ']:.2f} / {r['Baseline_MSE']:.4f} / {r['Baseline_MRE']:.3f}"
            proc_scores = f"{r['Processed_STOI']:.3f} / {r['Processed_PESQ']:.2f} / {r['Processed_MSE']:.4f} / {r['Processed_MRE']:.3f}"
            print(f"{r['Method']:<50} {r['Actual_SNR_dB']:<7.2f} {base_scores:<50} {proc_scores:<50}")
        print("="*165)
        
        # Line plots: Baseline vs Processed across SNR
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        fig.suptitle('Scenario 1: Real-world Noise - Metrics Scoring Across SNR Levels', fontsize=16, fontweight='bold')
        
        color_map = {
            'Deterministic (DTLN)': '#3b82f6',
            'Traditional-Manual (Spectral Subtraction)': '#f59e0b',
            'Traditional-Manual (Wiener Filter)': '#10b981',
            'Traditional-Library (noisereduce)': '#ef4444'
        }
        
        # STOI Comparison
        ax = axes[0, 0]
        for method in df_rw['Method'].unique():
            method_data = df_rw[df_rw['Method'] == method].sort_values('Actual_SNR_dB')
            ax.plot(method_data['Actual_SNR_dB'], method_data['Baseline_STOI'], 
                   linestyle='--', alpha=0.5, color=color_map.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data['Actual_SNR_dB'], method_data['Processed_STOI'], 
                   marker='o', label=method, linewidth=2.5, markersize=8,
                   color=color_map.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('STOI Score', fontweight='bold', fontsize=11)
        ax.set_title('STOI: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim([0, 1])
        ax.axhline(y=0.7, color='green', linestyle=':', alpha=0.5, label='Good threshold')
        
        # PESQ Comparison
        ax = axes[0, 1]
        for method in df_rw['Method'].unique():
            method_data = df_rw[df_rw['Method'] == method].sort_values('Actual_SNR_dB')
            ax.plot(method_data['Actual_SNR_dB'], method_data['Baseline_PESQ'],
                   linestyle='--', alpha=0.5, color=color_map.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data['Actual_SNR_dB'], method_data['Processed_PESQ'],
                   marker='s', label=method, linewidth=2.5, markersize=8,
                   color=color_map.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('PESQ Score', fontweight='bold', fontsize=11)
        ax.set_title('PESQ: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim([-0.5, 4.5])
        ax.axhline(y=2.5, color='green', linestyle=':', alpha=0.5, label='Good threshold')
        
        # MSE Comparison (log scale)
        ax = axes[1, 0]
        for method in df_rw['Method'].unique():
            method_data = df_rw[df_rw['Method'] == method].sort_values('Actual_SNR_dB')
            ax.plot(method_data['Actual_SNR_dB'], method_data['Baseline_MSE'],
                   linestyle='--', alpha=0.5, color=color_map.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data['Actual_SNR_dB'], method_data['Processed_MSE'],
                   marker='^', label=method, linewidth=2.5, markersize=8,
                   color=color_map.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('MSE Score (log scale)', fontweight='bold', fontsize=11)
        ax.set_title('MSE: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')
        ax.axhline(y=0.01, color='green', linestyle=':', alpha=0.5, label='Good threshold')
        
        # MRE Comparison
        ax = axes[1, 1]
        for method in df_rw['Method'].unique():
            method_data = df_rw[df_rw['Method'] == method].sort_values('Actual_SNR_dB')
            ax.plot(method_data['Actual_SNR_dB'], method_data['Baseline_MRE'],
                   linestyle='--', alpha=0.5, color=color_map.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data['Actual_SNR_dB'], method_data['Processed_MRE'],
                   marker='d', label=method, linewidth=2.5, markersize=8,
                   color=color_map.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('MRE Score', fontweight='bold', fontsize=11)
        ax.set_title('MRE: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim([0, 10])
        ax.axhline(y=0.5, color='green', linestyle=':', alpha=0.5, label='Good threshold')
        
        plt.tight_layout()
        plt.savefig('results/scenario1_realworld_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        # Bar chart: Method comparison
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle('Scenario 1: Average Scores by Method', fontsize=16, fontweight='bold')
        
        method_avg = df_rw.groupby('Method')[['Baseline_STOI', 'Processed_STOI', 
                                                'Baseline_PESQ', 'Processed_PESQ',
                                                'Baseline_MSE', 'Processed_MSE',
                                                'Baseline_MRE', 'Processed_MRE']].mean()
        
        metrics_to_plot = [
            ('STOI', ['Baseline_STOI', 'Processed_STOI'], axes[0, 0]),
            ('PESQ', ['Baseline_PESQ', 'Processed_PESQ'], axes[0, 1]),
            ('MSE', ['Baseline_MSE', 'Processed_MSE'], axes[1, 0]),
            ('MRE', ['Baseline_MRE', 'Processed_MRE'], axes[1, 1])
        ]
        
        for metric_name, cols, ax in metrics_to_plot:
            x = np.arange(len(method_avg))
            width = 0.35
            
            bars1 = ax.bar(x - width/2, method_avg[cols[0]], width, label='Baseline',
                          color='#94a3b8', edgecolor='black', linewidth=1)
            bars2 = ax.bar(x + width/2, method_avg[cols[1]], width, label='Processed',
                          color=[color_map.get(m, '#666') for m in method_avg.index],
                          edgecolor='black', linewidth=1)
            
            ax.set_xlabel('Method', fontweight='bold')
            ax.set_ylabel(f'{metric_name} Score', fontweight='bold')
            ax.set_title(f'Average {metric_name} Scores', fontweight='bold')
            ax.set_xticks(x)
            ax.set_xticklabels(method_avg.index, rotation=20, ha='right', fontsize=8)
            ax.legend()
            ax.grid(axis='y', alpha=0.3)
            
            # Add value labels
            for bars in [bars1, bars2]:
                for bar in bars:
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height,
                           f'{height:.3f}', ha='center', va='bottom', fontsize=8, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('results/scenario1_realworld_methods.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("✅ Scenario 1 visualizations saved")
    
    # ============================================================================
    # VISUALIZATIONS: SCENARIO 2 - SYNTHETIC
    # ============================================================================
    if len(synthetic_results) > 0:
        print(f"\n{'='*100}")
        print("📈 SCENARIO 2: SYNTHETIC VISUALIZATIONS")
        print("="*100)
        
        # Display scoring table (sample)
        print(f"\n{'Method':<50} {'Noise':<10} {'SNR':<7} {'Baseline (STOI/PESQ)':<25} {'Processed (STOI/PESQ)':<25}")
        print("="*120)
        for r in synthetic_results[:12]:
            base_scores = f"{r['Baseline_STOI']:.3f} / {r['Baseline_PESQ']:.2f}"
            proc_scores = f"{r['Processed_STOI']:.3f} / {r['Processed_PESQ']:.2f}"
            print(f"{r['Method']:<50} {r['Noise_Type']:<10} {r['Actual_SNR_dB']:<7.2f} {base_scores:<25} {proc_scores:<25}")
        if len(synthetic_results) > 12:
            print(f"... ({len(synthetic_results) - 12} more rows)")
        print("="*120)
        
        # Line plots: Similar to Scenario 1 but averaged across noise types
        fig, axes = plt.subplots(2, 2, figsize=(18, 12))
        fig.suptitle('Scenario 2: Synthetic Noise - Average Metrics Scoring Across SNR Levels', fontsize=16, fontweight='bold')
        
        color_map_syn = {
            'Stochastic (DTLN)': '#9333ea',
            'Traditional-Manual (Spectral Subtraction)': '#f59e0b',
            'Traditional-Manual (Wiener Filter)': '#10b981',
            'Traditional-Library (noisereduce)': '#ef4444'
        }
        
        # STOI
        ax = axes[0, 0]
        for method in df_syn['Method'].unique():
            method_data_base = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Baseline_STOI'].mean()
            method_data_proc = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Processed_STOI'].mean()
            ax.plot(method_data_base.index, method_data_base.values,
                   linestyle='--', alpha=0.5, color=color_map_syn.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data_proc.index, method_data_proc.values,
                   marker='o', label=method, linewidth=2.5, markersize=8,
                   color=color_map_syn.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('STOI Score', fontweight='bold', fontsize=11)
        ax.set_title('STOI: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim([0, 1])
        ax.axhline(y=0.7, color='green', linestyle=':', alpha=0.5)
        
        # PESQ
        ax = axes[0, 1]
        for method in df_syn['Method'].unique():
            method_data_base = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Baseline_PESQ'].mean()
            method_data_proc = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Processed_PESQ'].mean()
            ax.plot(method_data_base.index, method_data_base.values,
                   linestyle='--', alpha=0.5, color=color_map_syn.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data_proc.index, method_data_proc.values,
                   marker='s', label=method, linewidth=2.5, markersize=8,
                   color=color_map_syn.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('PESQ Score', fontweight='bold', fontsize=11)
        ax.set_title('PESQ: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim([-0.5, 4.5])
        ax.axhline(y=2.5, color='green', linestyle=':', alpha=0.5)
        
        # MSE (log scale)
        ax = axes[1, 0]
        for method in df_syn['Method'].unique():
            method_data_base = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Baseline_MSE'].mean()
            method_data_proc = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Processed_MSE'].mean()
            ax.plot(method_data_base.index, method_data_base.values,
                   linestyle='--', alpha=0.5, color=color_map_syn.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data_proc.index, method_data_proc.values,
                   marker='^', label=method, linewidth=2.5, markersize=8,
                   color=color_map_syn.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('MSE Score (log scale)', fontweight='bold', fontsize=11)
        ax.set_title('MSE: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')
        ax.axhline(y=0.01, color='green', linestyle=':', alpha=0.5)
        
        # MRE
        ax = axes[1, 1]
        for method in df_syn['Method'].unique():
            method_data_base = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Baseline_MRE'].mean()
            method_data_proc = df_syn[df_syn['Method'] == method].groupby('Actual_SNR_dB')['Processed_MRE'].mean()
            ax.plot(method_data_base.index, method_data_base.values,
                   linestyle='--', alpha=0.5, color=color_map_syn.get(method, '#666666'), linewidth=1.5)
            ax.plot(method_data_proc.index, method_data_proc.values,
                   marker='d', label=method, linewidth=2.5, markersize=8,
                   color=color_map_syn.get(method, '#666666'))
        ax.set_xlabel('SNR (dB)', fontweight='bold', fontsize=11)
        ax.set_ylabel('MRE Score', fontweight='bold', fontsize=11)
        ax.set_title('MRE: Baseline (dashed) vs Processed (solid)', fontweight='bold', fontsize=12)
        ax.legend(fontsize=8, loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim([0, 10])
        ax.axhline(y=0.5, color='green', linestyle=':', alpha=0.5)
        
        plt.tight_layout()
        plt.savefig('results/scenario2_synthetic_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        # Bar chart by noise type
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle('Scenario 2: Processed STOI Scores by Noise Type', fontsize=16, fontweight='bold')
        
        for idx, noise_type in enumerate(['Gaussian', 'White', 'Mixed']):
            ax = axes[idx]
            noise_data = df_syn[df_syn['Noise_Type'] == noise_type].groupby('Method')['Processed_STOI'].mean().sort_values(ascending=False)
            
            bars = ax.bar(range(len(noise_data)), noise_data.values,
                         color=[color_map_syn.get(m, '#666') for m in noise_data.index],
                         edgecolor='black', linewidth=1.5)
            ax.set_xticks(range(len(noise_data)))
            ax.set_xticklabels(noise_data.index, rotation=20, ha='right', fontsize=8)
            ax.set_ylabel('STOI Score', fontweight='bold')
            ax.set_title(f'{noise_type} Noise', fontweight='bold')
            ax.set_ylim([0, 1])
            ax.grid(axis='y', alpha=0.3)
            ax.axhline(y=0.7, color='green', linestyle='--', alpha=0.5)
            
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('results/scenario2_synthetic_by_noise.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("✅ Scenario 2 visualizations saved")
    
    print(f"\n{'='*100}")
    print("🎉 BATCH EVALUATION COMPLETED")
    print("="*100)
    print(f"✅ Scenario 1 (Real-world): {len(realworld_results)} experiments")
    print(f"✅ Scenario 2 (Synthetic): {len(synthetic_results)} experiments")
    print(f"\n📁 Files saved:")
    if len(realworld_results) > 0:
        print(f"   • results/scenario1_realworld_comparison.xlsx")
        print(f"   • results/scenario1_realworld_comparison.png")
        print(f"   • results/scenario1_realworld_methods.png")
    if len(synthetic_results) > 0:
        print(f"   • results/scenario2_synthetic_comparison.xlsx")
        print(f"   • results/scenario2_synthetic_comparison.png")
        print(f"   • results/scenario2_synthetic_by_noise.png")
    print(f"   • results/audio/*.wav (processed audio files)")

---

## 📥 Download Results
**One-click download of all generated files**

Downloads a ZIP package containing:
- ✅ Excel files (Scenario 1 & 2 with multiple sheets)
- ✅ Visualizations (performance charts & improvement graphs)
- ✅ Processed audio files (all experiments)
- ✅ Single experiment results (if executed)

In [ ]:
#@title 📦 Create & Download Results Package { display-mode: "form" }

#@markdown ---
#@markdown ### 📥 **Download Control**
download_results = False  #@param {type:"boolean"}

#@markdown ---
#@markdown ### 📝 **Package Contents:**
#@markdown - Excel: scenario1_realworld_comparison.xlsx
#@markdown - Excel: scenario2_synthetic_comparison.xlsx
#@markdown - Images: performance & improvement charts (PNG)
#@markdown - Audio: processed WAV files
#@markdown - Single: individual experiment results

if download_results:
    import zipfile
    from datetime import datetime
    
    print("📦 Creating download package...")
    
    # Create timestamp for unique filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_filename = f'dtln_evaluation_results_{timestamp}.zip'
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add Excel files
        for excel_file in ['results/scenario1_realworld_comparison.xlsx', 
                          'results/scenario2_synthetic_comparison.xlsx']:
            if os.path.exists(excel_file):
                zipf.write(excel_file, os.path.basename(excel_file))
                print(f"   ✓ {os.path.basename(excel_file)}")
        
        # Add visualizations
        for viz_file in ['results/scenario1_realworld_performance.png',
                        'results/scenario1_realworld_improvements.png',
                        'results/scenario2_synthetic_performance.png',
                        'results/scenario2_synthetic_improvements.png']:
            if os.path.exists(viz_file):
                zipf.write(viz_file, os.path.join('visualizations', os.path.basename(viz_file)))
                print(f"   ✓ visualizations/{os.path.basename(viz_file)}")
        
        # Add processed audio files
        audio_dir = 'results/audio'
        if os.path.exists(audio_dir):
            for audio_file in os.listdir(audio_dir):
                if audio_file.endswith('.wav'):
                    zipf.write(os.path.join(audio_dir, audio_file), 
                             os.path.join('audio', audio_file))
            print(f"   ✓ {len(os.listdir(audio_dir))} audio files")
        
        # Add single experiment results if exist
        if os.path.exists('results/spectrograms'):
            for spec_file in os.listdir('results/spectrograms'):
                if spec_file.startswith('single_'):
                    zipf.write(os.path.join('results/spectrograms', spec_file),
                             os.path.join('single_experiment', spec_file))
        
        if os.path.exists('results/metrics'):
            for metric_file in os.listdir('results/metrics'):
                if metric_file.startswith('single_'):
                    zipf.write(os.path.join('results/metrics', metric_file),
                             os.path.join('single_experiment', metric_file))
        
        if os.path.exists('outputs'):
            for output_file in os.listdir('outputs'):
                if output_file.startswith('single_'):
                    zipf.write(os.path.join('outputs', output_file),
                             os.path.join('single_experiment', output_file))
    
    file_size = os.path.getsize(zip_filename) / (1024 * 1024)
    print(f"\n✅ Package created: {zip_filename} ({file_size:.2f} MB)")
    print(f"\n📥 Downloading...")
    files.download(zip_filename)
    print(f"✅ Download started!")

---

## 📚 Evaluation Framework Documentation

### **Two Independent Scenarios:**

#### **🌍 Scenario 1: Real-world Comparison**
**Objective:** Evaluate performance on actual recorded noise

**Methods Evaluated:**
- Deterministic (DTLN) - 4 SNR levels
- Traditional-Manual (Spectral Subtraction) - 4 SNR levels
- Traditional-Manual (Wiener Filter) - 4 SNR levels
- Traditional-Library (noisereduce) - 4 SNR levels

**Total:** 16 experiments (4 methods × 4 SNR levels)

**Noise Source:** Single uploaded noise file (identical for all methods)

**SNR Levels:** -5, 0, 5, 10 dB

---

#### **🔬 Scenario 2: Synthetic Comparison**
**Objective:** Evaluate robustness across different noise types

**Methods Evaluated:**
- Stochastic (DTLN) - 3 noise types × 4 SNR levels = 12
- Traditional-Manual (Spectral Subtraction) - 3 noise types × 4 SNR levels = 12
- Traditional-Manual (Wiener Filter) - 3 noise types × 4 SNR levels = 12
- Traditional-Library (noisereduce) - 3 noise types × 4 SNR levels = 12

**Total:** 48 experiments (4 methods × 3 noise types × 4 SNR levels)

**Noise Types:** Gaussian, White, Mixed (synthetic)

**SNR Levels:** -5, 0, 5, 10 dB

---

### **Evaluation Metrics:**

| Metric | Description | Range | Better |
|--------|-------------|-------|--------|
| **STOI** | Short-Time Objective Intelligibility | 0-1 | Higher ↑ |
| **PESQ** | Perceptual Evaluation of Speech Quality | -0.5 to 4.5 | Higher ↑ |
| **MSE** | Mean Squared Error | 0-∞ | Lower ↓ |
| **MRE** | Mean Relative Error (masked) | 0-10 | Lower ↓ |

---

### **Method References:**

#### **1. Deterministic (DTLN)**
- **Paper:** Westhausen & Meyer (2020). "Dual-signal transformation LSTM network for real-time noise suppression"
- **Noise:** Fixed uploaded noise (deterministic)
- **Scenario:** Real-world only

#### **2. Stochastic (DTLN)**
- **Paper:** Same as Deterministic
- **Noise:** Random synthetic noise (stochastic)
- **Scenario:** Synthetic only

#### **3. Traditional-Manual (DSP)**
**Spectral Subtraction:**
- **Paper:** Boll (1979). "Suppression of acoustic noise in speech using spectral subtraction"
- **Parameters:** α=2.0 (over-subtraction), β=0.01 (spectral floor)

**Wiener Filter:**
- **Paper:** Lim & Oppenheim (1979). "Enhancement and bandwidth compression of noisy speech"
- **Method:** SNR-based gain estimation with minimum gain threshold (0.1)

**Noise Estimation:** First 10 frames (blind estimation)

#### **4. Traditional-Library (noisereduce)**
- **Library:** https://github.com/timsainb/noisereduce
- **Algorithm:** Spectral Gating (similar to Audacity)
- **Scenarios:** Both Real-world and Synthetic

---

### **Key Features:**

✅ **Single Experiment Mode:** Full visualization with before/after waveforms, spectrograms, overlay, and metrics

✅ **Batch Processing:** All combinations of Traditional-Manual methods (Spectral Subtraction AND Wiener Filter)

✅ **Fair Comparison:** All methods tested on identical noise conditions within each scenario

✅ **Download Package:** One-click download of all Excel files, visualizations, and audio files

✅ **Grand Total:** 64 experiments (16 real-world + 48 synthetic)

✅ **Auto-Export:** Results saved to Excel with multiple summary sheets

✅ **Blind Evaluation:** No ground truth leakage (noise estimated from audio)

---

### **Output Files Structure:**

```
dtln_evaluation_results_YYYYMMDD_HHMMSS.zip
├── scenario1_realworld_comparison.xlsx
│   ├── All_Results (detailed metrics)
│   ├── Summary_By_Method (mean & std)
│   └── Summary_By_SNR (aggregated)
├── scenario2_synthetic_comparison.xlsx
│   ├── All_Results (detailed metrics)
│   ├── Summary_By_Method (mean & std)
│   ├── Summary_By_Noise (per noise type)
│   └── Summary_By_SNR (aggregated)
├── visualizations/
│   ├── scenario1_realworld_performance.png
│   ├── scenario1_realworld_improvements.png
│   ├── scenario2_synthetic_performance.png
│   └── scenario2_synthetic_improvements.png
├── audio/
│   ├── realworld_*.wav (processed audio)
│   └── synthetic_*.wav (processed audio)
└── single_experiment/ (if single mode executed)
    ├── single_*_full.png (5-row visualization)
    ├── single_*_chart.png (metrics comparison)
    ├── single_*_before.wav
    └── single_*_after.wav
```

---

**📖 Usage Instructions:**

1. **Install Packages** - Run cell 2 to install dependencies
2. **Download Models** - Run cell 3 to get DTLN models
3. **Upload Audio** - Run cell 4 to upload clean speech and noise
4. **Load Functions** - Run cell 5 to initialize processing
5. **Single Test** (optional) - Run cell 7 for detailed single analysis
6. **Batch Evaluation** - Run cell 9 for comprehensive testing
7. **Download Results** - Run cell 11 to get all outputs

**🎯 Recommended Workflow:**
- Start with Single Experiment to verify setup
- Run Batch Evaluation for comprehensive results
- Download package for offline analysis

**⚡ Performance Notes:**
- Single experiment: ~5-10 seconds
- Batch evaluation: ~10-15 minutes (64 experiments)
- Results automatically saved during processing